# Dans ce travail nous avons utiliser **Tensorflow**, j'ai réimplementé la meme architecture que vous avez donnée et j'ai appliqué ce qui était demandé dans l'énnoncé

Pour la **troisieme** question j'ai changé le model comme suit:

**Convolution=> Convolution =>Pooling =>dropout=> Convolution=> Convolution =>Pooling =>dropout=> flatten=> dense=>dropout=>dense**

ce qui a permet d'améliorer les resultats d'accuracy et Loss  aussi j'ai ajouté utilisé.
Pour les parametres j'ai utilisé epoch= 15 batch_size=10 et learning rate =0.001

In [ ]:

import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
import numpy as np
from keras.utils import np_utils
from keras.optimizers import Adam

In [ ]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

In [ ]:
# convert from integers to floats
trainX = trainX.astype('float32')
testX = testX.astype('float32')

In [ ]:
mean =np.mean(trainX,axis=(0,1,2,3,))
std=np.std(trainX,axis=(0,1,2,3,))
trainX=(trainX-mean)/(std+1e-7)
testX=(testX-mean)/(std+1e-7)

In [ ]:
nClasses=10
trainY=np_utils.to_categorical(trainY,nClasses)
testY=np_utils.to_categorical(testY,nClasses)

In [ ]:
print(trainX.shape)
print(trainY.shape)

(50000, 32, 32, 3)
(50000, 10)


In [ ]:
input_shape=(32,32,3)

In [ ]:
def createModel():
  model=Sequential()
  model.add(Conv2D(32,(3,3),padding='same',activation = 'relu'))
  model.add(Conv2D(32,(3,3),activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Dropout(0.04))

  model.add(Conv2D(64,(3,3),padding='same',activation = 'relu'))
  model.add(Conv2D(64,(3,3),activation = 'relu'))
  model.add(MaxPooling2D(pool_size = (2, 2)))
  model.add(Dropout(0.04))


  model.add(Flatten())
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(nClasses,activation='softmax'))
  return model

In [ ]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	filename = sys.argv[0].split('/')[-1]
	pyplot.savefig(filename + '_plot.png')
	pyplot.close()
 

In [ ]:

model=createModel()

In [ ]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
input_shape = (trainX.shape)  
model.build(input_shape) 
model.summary()
#model.build(None,32,32,3) # `input_shape` is the shape of the input data
                         # e.g. input_shape = (None, 32, 32, 3)
#model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (50000, 32, 32, 32)       896       
_________________________________________________________________
conv2d_15 (Conv2D)           (50000, 30, 30, 32)       9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (50000, 15, 15, 32)       0         
_________________________________________________________________
dropout_10 (Dropout)         (50000, 15, 15, 32)       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (50000, 15, 15, 64)       18496     
_________________________________________________________________
conv2d_17 (Conv2D)           (50000, 13, 13, 64)       36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (50000, 6, 6, 64)        

In [ ]:
from keras.callbacks import Callback
class CustomCallBack(Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (epoch %5 ==0):
      print("just finished epoch",epoch)
      print("loss",logs.get('acc'))
      print("val_acc",logs.get('val_acc'))
      return

In [ ]:
batch_size=8
epoch=10
cc=CustomCallBack()
history=model.fit(trainX,trainY, batch_size=batch_size, epochs=epoch, validation_data=(testX,testY))

Epoch 1/10
6250/6250 [==============================] - 255s 41ms/step - loss: 1.7989 - accuracy: 0.3406 - val_loss: 1.1990 - val_accuracy: 0.5682
Epoch 2/10
6250/6250 [==============================] - 251s 40ms/step - loss: 1.1974 - accuracy: 0.5712 - val_loss: 1.0126 - val_accuracy: 0.6395
Epoch 3/10
6250/6250 [==============================] - 253s 41ms/step - loss: 0.9883 - accuracy: 0.6516 - val_loss: 0.8864 - val_accuracy: 0.6861
Epoch 4/10
6250/6250 [==============================] - 252s 40ms/step - loss: 0.8375 - accuracy: 0.7067 - val_loss: 0.7785 - val_accuracy: 0.7314
Epoch 5/10
6250/6250 [==============================] - 251s 40ms/step - loss: 0.7368 - accuracy: 0.7408 - val_loss: 0.7439 - val_accuracy: 0.7424
Epoch 6/10
6250/6250 [==============================] - 253s 40ms/step - loss: 0.6696 - accuracy: 0.7646 - val_loss: 0.7191 - val_accuracy: 0.7513
Epoch 7/10
6250/6250 [==============================] - 252s 40ms/step - loss: 0.6033 - accuracy: 0.7873 - val_loss: 0

Question 4 : Visualisation du filtre de la premiere couche